# More on finite differences

First a brief revision on vector spaces and norms.

## Vector spaces

A real *vector space* $V$ is a set endowed with two operations

1. *addition*: $(x, y) \in V \times V \to (x + y) \in V$
2. *scalar multiplication*: $(\alpha, x) \in \mathbb{R} \times V \to \alpha x \in V$

that obey the following axioms

1. associativity of addition: $x + (y + z) = (x + y) + z \text{ for all } x, y, z \in V$;
2. commutativity of addition: $x + y = y + x \text{ for all } x, y \in V$;
3. identity for addition: $\text{there exists } 0 \in V \text{ such that } x + 0 = x \text{ for all } x \in V$;
4. additive inverse: $\text{there exists an element } -x \in V \text{ such that }x + (-x) = 0 \text{ for all } x \in V$;
5. compatibility: $\alpha(\beta x) = (\alpha\beta)x \text{ for all } \alpha, \beta \in \mathbb{R}, x \in V$;
6. Identity for multiplication: $\text{there exists } 1 \in \mathbb{R} \text{ such that } 1 x = x \text{ for all } x \in V$;
7. Distribution of multiplication over addition: $a(x + y) = ax + ay$;
8. Distribution of addition over multiplication: $(a + b)x = ax + bx$;

This is a rather abstract definition, the familiar example is just Euclidean vectors in $\mathbb{R}^n$ (which is a vector space). In this case, addition is pointwise addition of components, and multiplication is multiplication of all components by a scalar.

As another example, the set $\operatorname{span} \{1, x, x^2\}$ of polynomials is a vector space where addition is the familiar addition, and multiplication is the familiar multiplication.

## Norms on vector spaces

A *norm* on a vector space $V$ is any mapping $\|\cdot\| : V \to \mathbb{R}$ that satisfies

$$
\begin{aligned}
\|x \| &\ge 0 && \text{for all } x \in V \text{ and } \|x\| = 0 \text{ if and only if } x = 0\\
\|\alpha x\| &= |\alpha|\|x\| && \text{for all } \alpha \in \mathbb{R}, x \in V\\
\|x + y\| &\le \|x\| + \|y\| && \text{for all } x, y \in V.
\end{aligned}
$$

Again, this is a rather abstract definition, it formalises an idea of distance. Thinking of our Euclidean space with $x \in \mathbb{R}^n$, the norm we are perhaps most familiar with is the 2-norm


$$
\| x \|_2 = \sqrt{\sum_i^n |x_i|^2}.
$$

This measure the distance in a straight line between two points in $\mathbb{R}^n$. The 1-norm (sometimes called the Manhattan norm) measures the distance if I can only walk along the unit vectors

$$
\|x\|_1 = \sum_i^n |x_i|.
$$

The $\infty$-norm (or max-norm) measures the maximum distance between the projection onto each of the unit vectors in turn

$$
\|x\|_\infty = \max_{0 < i \le n} |x_i|.
$$

In [ ]:
%matplotlib notebook
import numpy
from matplotlib import pyplot


def plot_norm_ball(p):
    x = numpy.linspace(-3, 3, 101)
    x, y = numpy.meshgrid(x, x)
    z = numpy.dstack([x, y])
    if p == "inf":
        p = r"\infty"
        z = numpy.linalg.norm(z, numpy.inf, axis=2)
    else:
        z = numpy.linalg.norm(z, p, axis=2)
    pyplot.figure()
    C = pyplot.contourf(x, y, z, numpy.linspace(0, 1, 2), cmap=pyplot.cm.coolwarm)
    
    pyplot.contour(x, y, z, numpy.linspace(0, 1, 2), colors='k')
    pyplot.xlabel("x")
    pyplot.ylabel("y")
    pyplot.title(f"Unit ball in ${p}$-norm")
    pyplot.gca().set_aspect("equal")
    return pyplot.gcf()

In [ ]:
for p in [1, 2, 4, "inf"]:
    plot_norm_ball(p)

The unit ball for the 4-norm was advocated by [Piet Hein](https://en.wikipedia.org/wiki/Piet_Hein_(scientist)) as a pleasing shape for things like coffee tables and buildings.

## Matrix representation of operators

We saw briefly that we can represent finite difference operators as matrices. Let's look at this in a bit more detail.

To do this, we need to provide an ordering of all of the degrees of freedom (dofs) in our finite difference discretisation. In one dimension, we order the points in the domain from left to right and use a single index:

$$
x_0 < x_1 < \dots < x_{n-1}
$$

and so we have a single index for all the points $i = [0, 1, \dots, n-1]$. We can therefore represent our function $u(x)$ discretised at the points $\{x_i\}$ as a vector in $\mathbb{R}^n$

$$
U = \begin{bmatrix} u_0 \\ u_1 \\ \vdots \\ u_{n-1} \end{bmatrix}
$$

and similarly with the right hand side $f(x)$. The differencing operators *combine* entries from $U$ linearly to produce a new vector $D U$. Since this operation is linear, we can represent it as a matrix

$$
D : \mathbb{R}^n \to \mathbb{R}^n
$$

which takes in a vector $U$ and spits out a new vector representing the action of the differencing operator on $U$.

For example, the left-looking operator $D_- u_i = \frac{u_i - u_{i-1}}{h}$ uses, at each point $i$ values from points $i$ and $i-1$. On a grid with 4 points, this can be represented as the matrix

$$
D_- = \frac{1}{h}
\begin{bmatrix}
1 & 0 & 0 & 0\\
-1 & 1 & 0 & 0\\
0 & -1 & 1 & 0\\
0 & 0 & -1 & 1
\end{bmatrix}.
$$

Similarly, the centered difference approximation of $\frac{\text{d}^2}{\text{d} x^2}$, $D^2 u_i = \frac{u_{i+1} - 2u_i + u_{i-1}}{h^2}$ can be written

$$
D^2 = \frac{1}{h^2}
\begin{bmatrix}
-2 & 1 & 0 & 0\\
1 & -2 & 1 & 0\\
0 & 1 & -2 & 1\\
0 & 0 & 1 & -2
\end{bmatrix}.
$$

### "Matrix-free" implementation

If we only never need to apply the differencing operator, it might make sense (memory or efficiency, for example) to just provide a function which computes the matrix-vector multiplication without storing the matrix. Let's see this in action.

In [ ]:
%matplotlib notebook
import numpy

def dminus(u, h):
    n, = u.shape
    du = numpy.zeros_like(u)
    for i in range(n):
        if i == 0:
            du[i] = 1/h * u[i]
        else:
            du[i] = 1/h * (u[i] - u[i-1])
    return du

def dminusop(u, h):
    n, = u.shape
    D = numpy.eye(n) - numpy.diag(numpy.full(n-1, 1), k=-1)
    D *= 1/h
    return D

In [ ]:
n = 10
u = numpy.random.rand(n)
h = 1/n
dminus(u, h)

In [ ]:
D = dminusop(u, h)

In [ ]:
D @ u

## 2D finite differences
### Indexing matrix representation in 2D
### Sparse matrices

## CFL conditions